# Logarithms
### Various numerical methods to calculate logarithms 


#### Standard power series
Start with the normal power series for $ln(1+t)$ (for $t\in(-1,1)$)

$\ln(1+t) = \sum_{n=1}^{\infty} \frac{(-1)^{n+1} t^n}{n} = t - \frac{t^2}{2} + \frac{t^3}{3} - \frac{t^4}{4} + \cdots  = t + \mathcal{O}(t^2)$

#### More cleverer power series:
Trick to increase to quadratic convergence: subtract all odd terms:

$$\begin{align}
\ln\left(\frac{1+t}{1-t} \right) = ln(1+t) - \ln(1-t) &= t - \frac{t^2}{2} + \frac{t^3}{3} - \frac{t^4}{4} +  \frac{t^5}{5} + \cdots \\
                     &+t + \frac{t^2}{2} + \frac{t^3}{3} + \frac{t^4}{4} +  \frac{t^5}{5} + \cdots \\
                    &= 2\left( t +          \frac{t^3}{3}              + \frac{t^5}{5} + \cdots \right) \\
                    &= 2t + \mathcal{O}(t^3)
\end{align}$$
Then set $x = \frac{1+t}{1-t}$, or, $t = \frac{x-1}{x+1} $, so a faster series for $\ln(x)$ is:
$$\begin{align}
\ln(x) = 2\left(\frac{x-1}{x+1}\right) + 2\frac{1}{3}\left(\frac{x-1}{x+1}\right)^3   + 2\frac{1}{5}\left(\frac{x-1}{x+1}\right)^5 + \cdots
\end{align}$$



#### Newton iteration on $f(y) = \exp(y) - x$
Note: Ensure that y > 0 to avoid the near zero derivative. This is done by setting x > 0 with the Factor trick.

$$\begin{align}
    y_{n+1} &= y_n - \frac{\exp(y_n) - x}{\exp(y_n)} \\ 
            &= y_n - (1 - x \exp(-y_n))
\end{align}$$

For more efficient computation: Use the approximate Newton method by approximating the Exp with its power series

$$\begin{align}
    \exp(-y) &\approx 1 - y + \frac{1}{2}y^2 - \frac{1}{3}y^3 + ... \\ 
            &\Rightarrow \\
    y_{n+1} &= y_n - 1 + x\left( 1 - y + \frac{1}{2}y_n^2 - \frac{1}{3}y_n^3 \right)  \\ 
\end{align}$$





In [105]:
from math import log, exp, sqrt # Only for accuracy calculation
import matplotlib.pyplot as plt
import numpy as np

EPS = 5e-16 # Maximal precision

In [106]:
def FactorTrick(x, xmin = .5, xmax=1.5):
    """ 
        Outputs xr and k such that 
            x = xr * 2^k with xr in xmin, xmax

        This allows for
        log(x) = log(xr) + k log(2), with xr in a better range for convergence
    """
    assert xmin * 2 <= xmax
    k = 0
    while x >= xmax:
        x = x/2
        k+=1
    while x <= xmin:
        x *= 2
        k -= 1
    return x, k

In [120]:

def LogSeries(x, max_iter=22):
    """ Series calculation for Logarithm."""
    # Divisions: 1 + N
    # Mulitply: 1 + 2N
    info = {
        'iters'   : 0,
        'success' : False,
        'flops'   : 2,  # 2 flops for setup
    }
    y = 0 # running sum
    q = (x-1)/(x+1) # Quotient
    qq = q*q
    for n in range(max_iter):
        add = 2/(2*n+1) * q # 2 flops
        if abs(add) < EPS:
            info['success'] = True
            break
        y += add
        q *= qq # 1 flop
        
        info['flops'] += 3 # 3 flops per iteration
        info['iters'] += 1

    info['res_iter'] = abs(add)
    info['res_actual'] = log(x) - y
    info['accuracy'] = float('inf') if log(x) == y else -log(abs(log(x) - y),10) # Number of correct digits in base-10
    return y, info



In [ ]:
def LogNewton(x, max_iter=22, y = None, exp_approx = 4, good_initial_guess = False):
    """ Approximate Newton iteration for Logarithm
        Zeros for the function:
            exp(y) - x
        Newton iteration with initial guess t0
            y1 = y0 - (exp(y0) - x) / exp(y0)
               = y0 - x exp(-y0) 
        Approximate the exp function with its series
            exp(-y0) = 1 -y0 + 1/2 y0^2 - 1/6 y0^3 + ... (3 flops per iteration)   
    """
    info = {
        'iters'   : 0,
        'success' : False,
        'flops'   : 0,  # 0 flops for setup
        'res_iter': float('inf'),
    }
    if x < 1:
        info['warning'] = 'x<1. Poor performance on newton iteration.'
    if y is None: # No initial guess provided 
        y = 1  #Zeroth order guess
        if good_initial_guess: # Use first term of series approximation
            y = 2*(x-1)/(x+1)
            info['flops'] += 1 # One flop used for initial guess.
    for i in range(max_iter):
        # Calculate approximate exponential series
        exp_y = 1
        ynext = 1   # powers of y
        for j in range(1,exp_approx+1):
            ynext *= y / -j # Increase exponent, increase factorial denominator, swap sign
            exp_y += ynext
            info['flops'] += 2 # 2 flops per exp sum iteration
            
        y1 = y - 1 + x*exp_y # Flops: 1
        info['flops'] += 1 # 1 flop per newton iteration
        info['iters'] += 1
        info['res_iter'] = abs(y1-y)
        if info['res_iter'] < EPS:
            info['success'] = True
            break
        y = y1

    info['res_actual'] = log(x) - y
    info['accuracy'] = float('inf') if log(x) == y else -log(abs(log(x) - y),10) # Number of correct digits in base-10

    return y, info



In [ ]:
def LogSimpson(x, max_iters=22):
    """ Integrate 1/t in 0<t<x 
        Using Simpson's rule: (https://en.wikipedia.org/wiki/Simpson%27s_rule)
             b           b-a             a+b
            / f(x) dx ~= --- [ f(a) +4 f(---) + f(b) ]
           a              6               2
    """
    assert x > 1
    max_iters = max(1, max_iters)
    info = {
        'iters'   : max_iters, # Always run full segmentation, no break condition
        'success' : True, # no break condition, iteration always succeeds
        'flops'   : 1,  # 1 flop for setup
    }
    t = 1
    dt = (x-1) / max_iters # = b-a, segement length (flops: 1)
    dt2 = dt/2            # = (b-a) /2, segment midpoint
    dt6 = dt/6            # = (b-a) /6
    y=0         # Running sum
    fa = 1      # =f(a)
    for i in range(max_iters):
        fm = 1/(t+dt2) # = f((a+b)/2), iterand at midpoint
        fb = 1/(t+dt)  # = f(b), iterand at right edge
        y += dt6*(fa + 4*fm + fb) # Add Simpson's rule on segment (a,b)
        # Shift over to next segment
        t+=dt
        fa = fb # Iterand at right becomes iterand at left 
        info['flops'] += 4 # two divisions, 2 multiplies

    info['res_actual'] = log(x) - y
    info['accuracy'] = float('inf') if log(x) == y else -log(abs(log(x) - y),10) # Number of correct digits in base-10

    return y, info


In [110]:
def AGM(a,g, max_iters = 12):
    info = {
        'iters':0,
        'success': False
    }
    for i in range(max_iters):
        a,g = (a+g)/2, sqrt(a*g)
        info['iters'] +=1
        if abs(a-g) < EPS:
            info['success'] = True
            break
    info['res_iter'] = abs(a-g)
    return a, info

AGM(1,1.2)



(1.0977213762523852,
 {'iters': 4, 'success': True, 'res_iter': 2.220446049250313e-16})